TODO list:

- wire all 3 servos reliably
- test communication with all servos at once
- create mounts on servo arms (small metal piece)

In [34]:
import serial
import numpy as np
from collections import defaultdict
import time
import threading

In [68]:
class ServoSerial:
    def __init__(self, num_servos, port='/dev/ttyACM1', baudrate=115200, offset=0.0):
        self.offset = 0.05
        self.min_angle = -np.pi/4.
        self.max_angle = np.pi/4.
        
        self.num_servos = num_servos
        self.serial = serial.Serial(port, baudrate=baudrate)
        self.curr_angles = defaultdict(float)
        self.write_lock = threading.RLock()
        for i in xrange(self.num_servos):
            self.set_angle(i, self.curr_angles[i])
        
    def is_valid_servo_num(self, servo_num):
        is_valid = 0 <= servo_num < self.num_servos
        if not is_valid:
            print('WARNING: servo_num {0} not in [{1}, {2})'.format(servo_num, 0, self.num_servos))
        return is_valid
    
    def is_valid_angle(self, angle):
        is_valid = self.min_angle <= angle <= self.max_angle
        if not is_valid:
            print('WARNING: angle {0} not in [{1},{2}]'.format(angle, self.min_angle, self.max_angle))
        return is_valid
    
    def get_angle(self, servo_num):
        if not self.is_valid_servo_num(servo_num):
            return
        
        return self.curr_angles[servo_num]
        
    def set_angle(self, servo_num, des_angle, speed=None, fs=100.):
        """
        :param servo_num: servo to command
        :param des_angle: desired angle (radians)
        :param speed: if None, go as fast as possible. Else rad/s
        :param fs: if speed is not None, command discretization
        """
        if not (self.is_valid_servo_num(servo_num) and self.is_valid_angle(des_angle)):
            return
        
        if speed is None:
            self.write_lock.acquire()
            self.serial.write('{0}{1}\n'.format(servo_num, self.offset+des_angle))
            self.write_lock.release()
        else:
            curr_angle = self.curr_angles[servo_num]
            sign = 1 if des_angle > curr_angle else -1
            angles = np.r_[curr_angle:des_angle:sign*speed/fs]
            for angle in angles:
                self.write_lock.acquire()
                self.serial.write('{0}{1}\n'.format(servo_num, self.offset+angle))
                self.write_lock.release()
                self.curr_angles[servo_num] = angle
                time.sleep(1/fs)
                
        self.curr_angles[servo_num] = des_angle
        
servo = ServoSerial(3)

In [73]:
curr_angle = s.get_angle(0)
des_angle = 0.
speed = 0.5
expected_time = abs(des_angle - curr_angle)/speed

start = time.time()
servo.set_angle(2, des_angle, speed=speed)
end = time.time()

print('Expected time: {0}'.format(expected_time))
print('Elapsed: {0}'.format(end - start))

Expected time: 0.0
Elapsed: 0.0001540184021


In [74]:
class ServoSDR:
    def __init__(self, servo_num, servo, sdr):
        self.servo_num = servo_num
        self.servo = servo
        self.sdr = sdr
        
        self.speed = np.pi/4.
        self.run_flag = False
        self.is_stopped = False
        
    def start(self, speed=None):
        if self.run_flag:
            return
        
        if speed is not None:
            self.speed = speed
        
        self.run_flag = True
        self.thread = threading.Thread(target=self.run, args=())
        self.thread.daemon = True                            # Daemonize thread
        self.thread.start()                 
    
    def stop(self):
        if not self.run_flag:
            return
        
        self.run_flag = False
        
        while not self.is_stopped:
            time.sleep(0.2)
        self.is_stopped = False
    
    def run(self):
        # TODO: add SDR reading
        while self.run_flag:
            self.servo.set_angle(self.servo_num, self.servo.min_angle, speed=self.speed)
            if not self.run_flag:
                break
            self.servo.set_angle(self.servo_num, self.servo.max_angle, speed=self.speed)
            
        self.is_stopped = True
            
servo_sdr0 = ServoSDR(0, servo, None) # TODO: SDR
servo_sdr1 = ServoSDR(1, servo, None)
servo_sdr2 = ServoSDR(2, servo, None)

In [75]:
servo_sdr0.start()
servo_sdr1.start()
#servo_sdr2.start()

In [ ]:
servo_sdr0.stop()
servo_sdr1.stop()
#servo_sdr2.stop()